<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


In [10]:
!pip install openai --quiet

In [11]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [12]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [13]:
def call(system_prompt: str, message: str) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [14]:
def marketing_campaign_agent(product : str):
  """
     <PRODUCT>{product}</PRODUCT>

     Your task is to create an compelling marketing campaign for the product provided between the <product> tag.
  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [18]:
def ready_validation_agent(marketing_text : str) -> bool:
  """

    <CONTENT>{text}</CONTENT>
    I have asked you to create a marketing text for a product  earlier. You can find the text between the <CONTENT> tags.

    Is this the final version of the marketing text and ready to publish or can we improve it?

     Your task is to answer with
     - TRUE if this is the final version of the marketing text and ready to publish.
     - FALSE if this is not the final version of the marketing text and we need to improve it.

     Only answer with TRUE or FALSE. add nothing else to your response.

    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(text=marketing_text) )
  print (result)
  return result.strip().lower() == "true"

In [31]:
def refinements_agent(product : str, marketing_text : str) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  I have asked you to create a marketing text for a product earlier. You can find the text between the <CONTENT> tags.

  This marketing text is not final and not yet ready for publishing.

  Your task is to refine the marketing text provided betwen the <CONTENT> tag with the follwing refinements. Don't use any other sources.

  ### OUTPUT
  Please output a  list of improvements you have found.

  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text) )

In [42]:
def refine_agent(product : str, marketing_text : str, refinements : str ) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  <REFINE>{refinements}</REFINE>

  I have asked you to create a marketing campaign for a product earlier. You can find the markeging campaign between the <CONTENT> tags.

  Your task is to add the refinements provided between <REFINE> tags to that marketing campaign .



  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text, refinements=refinements) )

In [47]:
product = "iPhone 15"

iteration=1
marketing_text = marketing_campaign_agent(product)
while not ready_validation_agent(marketing_text) and iteration<5:
  print (f"ITERATION: {iteration}" )
  refinements = refinements_agent(product, marketing_text)
  next_text_iteration = refine_agent(product, marketing_text, refinements)
  print (refinements)
  iteration+=1
  marketing_text = next_text_iteration


FALSE


TypeError: can only concatenate str (not "int") to str

In [44]:
print (next_text_iteration)


**Refined Marketing Text:**

<PRODUCT>iPhone 15</PRODUCT>

<CONTENT>
**Marketing Campaign: "Unleash Your Potential" - iPhone 15 Launch**
 
**Campaign Objective:** Create awareness, generate excitement, and drive sales of iPhone 15 among tech-savvy consumers worldwide.

**Target Audience:** Tech enthusiasts, mobile device users, and individuals seeking a premium smartphone experience.

**Key Messaging:

* Discover the latest innovation from Apple - iPhone 15, empowering you to unlock new possibilities with cutting-edge technology and stunning design.
* Unlock your world with iPhone 15, featuring groundbreaking camera capabilities, lightning-fast performance, and an immersive display that redefines mobile entertainment.

**Marketing Strategies:**

1. **Teaser Campaign:**
Release a series of eye-opening ads on social media platforms (Instagram, TikTok, Facebook) showcasing the iPhone 15's innovative features.
Create a hashtag (#EmpowerYourWorld) and encourage users to share their own stor

In [45]:
print (marketing_text)
print ("###### critique")
print (is_finished)
print (refinements)
print ("###### next text iteration")
print (next_text_iteration)

**Marketing Campaign: "Empower Your World" - iPhone 15 Launch**

**Campaign Objective:** Drive awareness, generate buzz, and drive sales of iPhone 15 among tech-savvy consumers worldwide.

**Target Audience:** Tech enthusiasts, mobile device users, and individuals looking for a premium smartphone experience.

**Key Messaging:**

* Introducing the latest innovation from Apple - iPhone 15, redefining the smartphone experience with cutting-edge technology and sleek design.
* Unlock your world with iPhone 15, featuring groundbreaking camera capabilities, lightning-fast performance, and an immersive display.

**Marketing Strategies:**

1. **Teaser Campaign:**
	* Release a series of cryptic ads on social media platforms (Instagram, TikTok, Facebook) showcasing a fragmented smartphone interface, hinting at the iPhone 15's advanced features.
	* Create a hashtag (#EmpowerYourWorld) and encourage users to share their own mobile moments, using the iPhone 15 as their device of choice.
2. **Influen